In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as DateTime, timedelta as TimeDelta

In [2]:
docu = pd.ExcelFile('Covid Chile V2 PARA PRUEBAS.xlsx')

In [3]:
docuConfirma = pd.read_excel(docu,  sheet_name="COVID_CL_CONFIRMA")
docuMuertes = pd.read_excel(docu,  sheet_name="COVID_CL_MUERTE")
docuRecuperados = pd.read_excel(docu,  sheet_name="COVID_CL_RECUPERA")

In [4]:
docuConfirma.dropna(axis = 0, how = 'all', inplace = True)
if(docuConfirma.columns[0] == 'Unnamed: 0'):
    docuConfirma.columns = docuConfirma.iloc[0]
    docuConfirma = docuConfirma.iloc[1:,].reindex()
    
docuMuertes.dropna(axis = 0, how = 'all', inplace = True)
if(docuMuertes.columns[0] == 'Unnamed: 0'):
    docuMuertes.columns = docuMuertes.iloc[0]
    docuMuertes = docuMuertes.iloc[1:,].reindex()
    
docuRecuperados.dropna(axis = 0, how = 'all', inplace = True)
if(docuRecuperados.columns[0] == 'Unnamed: 0'):
    docuRecuperados.columns = docuRecuperados.iloc[0]
    docuRecuperados = docuRecuperados.iloc[1:,].reindex()

In [5]:
fechaMinima = docuConfirma.Fecha.min()
fechaMaxima = docuConfirma.Fecha.max() + TimeDelta(days=5)

listaFechas = []
listaFechas = pd.date_range(start=fechaMinima,end=fechaMaxima)

df3 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"], "Codigo comuna" : docuConfirma["ID_Comuna"]})
res2 = df3.groupby(["Region","Comuna", "Codigo comuna"]).sum().reset_index()

In [6]:
lista = []
for Comuna in res2["Comuna"]:
    for Fecha in listaFechas:
        lista.append(Fecha)
        
res3 = res2.loc[np.repeat(res2.index.values, len(listaFechas))].reset_index(drop = True)

res3['Fecha'] = lista
res3['Casos Diarios'] = 0
res3['Fallecidos Diarios'] = 0
res3['Recuperados Diarios'] = 0

In [7]:
df4 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"],"Fecha" : docuConfirma["Fecha"],"Codigo comuna" : docuConfirma["ID_Comuna"] , "Casos Diarios" : 0})

resConfirma4 = df4.groupby(["Region","Comuna","Fecha",'Codigo comuna']).count().reset_index()
resConfirma4.insert(4, "Fallecidos Diarios", 0, True)
resConfirma4.insert(5, "Recuperados Diarios", 0, True)
resConfirma5 = pd.concat([ res3, resConfirma4 ])
resConfirma6 = resConfirma5.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

In [8]:
df5 = pd.DataFrame({"Region" : docuMuertes["Region"], "Comuna" : docuMuertes["Comuna"], "Codigo comuna" : docuMuertes["ID_Comuna"] ,"Fecha" : docuMuertes["Fecha"], "Fallecidos Diarios" : 0})
resMuerte4 = df5.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resMuerte4.insert(4, "Casos Diarios", 0, True)
resMuerte4.insert(6, "Recuperados Diarios", 0, True) 
resMuerte5 = pd.concat([ resConfirma6 ,  resMuerte4])
resMuerte6 = resMuerte5.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

In [35]:
df6 = pd.DataFrame({"Region" : docuRecuperados["Region"], "Comuna" : docuRecuperados["Comuna"], "Codigo comuna" : docuRecuperados["ID_Comuna"] ,"Fecha" : docuRecuperados["Fecha"], "Recuperados Diarios" : 0})
resRecupera4 = df6.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resRecupera4.insert(4, "Casos Diarios", 0, True)
resRecupera4.insert(5, "Fallecidos Diarios", 0, True) 
resTotal = pd.concat([ resMuerte6 ,  resRecupera4])
resTotal = resTotal.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

In [36]:
#resTotal["Casos Acumulados"] = resTotal['Casos Diarios']
resTotal.insert(5, "Casos Acumulados", resTotal['Casos Diarios'], True)
resTotal.insert(6, "Muertes", resTotal['Fallecidos Diarios'], True)
resTotal.insert(7, "Recuperados", resTotal['Recuperados Diarios'], True) 
resTotal.insert(8, "Casos Activos", resTotal['Casos Acumulados'], True)
resTotal['Dia'] = 0
resTotal['on-off'] = 'OFF'
inicial = True

for i in range(1, len(resTotal)):
    if(resTotal.loc[i, 'Comuna'] == resTotal.loc[i-1, 'Comuna']):
        resTotal.loc[i, 'Casos Acumulados'] = resTotal.loc[i, 'Casos Acumulados'] + resTotal.loc[i-1, 'Casos Acumulados']
        resTotal.loc[i, 'Muertes'] = resTotal.loc[i, 'Muertes'] + resTotal.loc[i-1, 'Muertes']
        resTotal.loc[i, 'Recuperados'] = resTotal.loc[i, 'Recuperados'] + resTotal.loc[i-1, 'Recuperados']
        resTotal.loc[i, 'Casos Activos'] = (resTotal.loc[i, 'Casos Acumulados'] - resTotal.loc[i, 'Muertes']) - resTotal.loc[i, 'Recuperados']
        if(resTotal.loc[0, 'Casos Diarios'] > 0):
            resTotal.loc[0, 'Dia'] = 1
        if(resTotal.loc[i, 'Casos Diarios'] > 0):
            inicial = False
            resTotal.loc[i, 'on-off'] = 'START'
        if(inicial == False):
            resTotal.loc[i, 'Dia'] = 1 + resTotal.loc[i-1, 'Dia']
            if(resTotal.loc[i-1, 'on-off'] == 'OFF'):
                resTotal.loc[i, 'on-off'] = 'START'
            else: 
                if(resTotal.loc[i-1, 'on-off'] == 'START' or resTotal.loc[i-1, 'on-off'] == 'ON'):
                    resTotal.loc[i, 'on-off'] = 'ON'
            if(resTotal.loc[i, 'Fecha'] > docuConfirma.Fecha.max()):
                resTotal.loc[i, 'on-off'] = 'OUT TIME' 
    else:
        inicial = True
        if(resTotal.loc[i, 'Casos Diarios'] > 0):
            resTotal.loc[i, 'Dia'] = 1
            resTotal.loc[i, 'on-off'] = 'START'
            inicial = False

In [37]:
resTotal

,Region,Comuna,Fecha,Codigo comuna,Casos Diarios,Casos Acumulados,Muertes,Recuperados,Casos Activos,Fallecidos Diarios,Recuperados Diarios,Dia,on-off
0,Antofagasta,Antofagasta,2020-03-03,2101,0,0,0,0,0,0,0,0,OFF
1,Antofagasta,Antofagasta,2020-03-04,2101,0,0,0,0,0,0,0,0,OFF
2,Antofagasta,Antofagasta,2020-03-05,2101,0,0,0,0,0,0,0,0,OFF
3,Antofagasta,Antofagasta,2020-03-06,2101,0,0,0,0,0,0,0,0,OFF
4,Antofagasta,Antofagasta,2020-03-07,2101,0,0,0,0,0,0,0,0,OFF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23304,Ñuble,Ñiquén,2020-05-13,16303,0,3,1,0,2,0,0,36,OUT TIME
23305,Ñuble,Ñiquén,2020-05-14,16303,0,3,1,0,2,0,0,37,OUT TIME
23306,Ñuble,Ñiquén,2020-05-15,16303,0,3,1,0,2,0,0,38,OUT TIME
23307,Ñuble,Ñiquén,2020-05-16,16303,0,3,1,0,2,0,0,39,OUT TIME


In [38]:
resTotal['Max Dia'] = 0

In [39]:
fechaMinima = docuConfirma.Fecha.min()
fechaMaxima = docuConfirma.Fecha.max()

listaFechas2 = []
listaFechas2 = pd.date_range(start=fechaMinima,end=fechaMaxima)

In [40]:
primer = resTotal[resTotal.groupby(['Region', 'Comuna'])['Casos Diarios'].cumsum() != 0].reset_index(drop = True)

In [41]:
tercero = primer[primer.Fecha < fechaMaxima].groupby(['Region', 'Comuna'])['Casos Diarios'].agg([('start','first'),('end','last'),('Max Dia','size')]).reset_index()
tercero['Max Dia'] = tercero['Max Dia']+1
tercero

,Region,Comuna,start,end,Max Dia
0,Antofagasta,Antofagasta,3,28,55
1,Antofagasta,Calama,2,3,60
2,Antofagasta,María Elena,1,0,30
3,Antofagasta,Mejillones,1,5,41
4,Antofagasta,No Informado,1,0,13
...,...,...,...,...,...
297,Ñuble,San Fabián,1,0,46
298,Ñuble,San Ignacio,1,0,41
299,Ñuble,San Nicolás,1,0,42
300,Ñuble,Yungay,2,0,46


In [42]:
fechaMaxima

datetime.datetime(2020, 5, 12, 0, 0)

In [43]:
#Arreglar fecha actual
#quitar hora
dt = DateTime.now()
dt = dt.replace(hour=0, minute=0, second=0, microsecond=0) 
dt

datetime.datetime(2020, 5, 13, 0, 0)

In [44]:
c = 0
for i in range(0, len(resTotal)):
    for j in range(c, len(tercero)):
        if(resTotal.loc[i, 'Comuna'] == tercero.loc[j, 'Comuna']):
            resTotal.loc[i, 'Max Dia'] = tercero.loc[j , 'Max Dia']
            c = j
            break

In [45]:
resTotal

,Region,Comuna,Fecha,Codigo comuna,Casos Diarios,Casos Acumulados,Muertes,Recuperados,Casos Activos,Fallecidos Diarios,Recuperados Diarios,Dia,on-off,Max Dia
0,Antofagasta,Antofagasta,2020-03-03,2101,0,0,0,0,0,0,0,0,OFF,55
1,Antofagasta,Antofagasta,2020-03-04,2101,0,0,0,0,0,0,0,0,OFF,55
2,Antofagasta,Antofagasta,2020-03-05,2101,0,0,0,0,0,0,0,0,OFF,55
3,Antofagasta,Antofagasta,2020-03-06,2101,0,0,0,0,0,0,0,0,OFF,55
4,Antofagasta,Antofagasta,2020-03-07,2101,0,0,0,0,0,0,0,0,OFF,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23304,Ñuble,Ñiquén,2020-05-13,16303,0,3,1,0,2,0,0,36,OUT TIME,35
23305,Ñuble,Ñiquén,2020-05-14,16303,0,3,1,0,2,0,0,37,OUT TIME,35
23306,Ñuble,Ñiquén,2020-05-15,16303,0,3,1,0,2,0,0,38,OUT TIME,35
23307,Ñuble,Ñiquén,2020-05-16,16303,0,3,1,0,2,0,0,39,OUT TIME,35


In [46]:
resTotal.to_csv("FINAL.csv", index=False)